In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import joblib
from sklearn.model_selection import KFold

In [3]:
df = pd.read_csv(r"/kaggle/input/feature-common-language/audio_features_partial.csv")
df.head()

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,spectral_rolloff,zero_crossing_rate,mfcc_1,mfcc_2,mfcc_3,mfcc_4,...,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,label,file_path
0,0.712224,0.055740,2989.050985,2193.800068,5666.294643,0.275144,-202.32579,27.614292,4.094784,5.301181,...,4.752774,-0.985637,-6.752584,-4.679379,-5.478848,-0.866508,-1.919669,-0.634521,Arabic,/kaggle/input/preprocess-common-language/proce...
1,0.598403,0.074004,2372.315827,2065.561020,4795.649510,0.178041,-137.41476,59.931843,5.077963,-5.712012,...,-12.469353,-2.134825,-10.989368,-1.460541,-4.485021,-0.408789,-8.211143,-5.170048,Arabic,/kaggle/input/preprocess-common-language/proce...
2,0.756316,0.046051,3274.178654,2196.474265,5870.572917,0.353054,-233.02031,22.691550,10.057923,3.829097,...,-1.657243,-5.409642,-4.017134,-6.744406,-1.697630,-0.387302,0.829549,1.292110,Arabic,/kaggle/input/preprocess-common-language/proce...
3,0.588983,0.061376,1948.418292,2049.242741,4186.921296,0.137682,-199.04490,80.806870,31.451380,-1.297673,...,-4.202263,0.065943,-9.312079,-7.164060,-3.082040,-8.046175,-3.083879,-2.018449,Arabic,/kaggle/input/preprocess-common-language/proce...
4,0.647222,0.069638,1705.618989,1824.714129,3730.709877,0.095540,-325.47556,92.373820,17.725632,31.867613,...,3.169903,4.538502,-1.073114,-1.204524,-0.108214,-4.803460,-2.882802,-1.455632,Arabic,/kaggle/input/preprocess-common-language/proce...


In [4]:
df.duplicated().sum()

0

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder with your actual labels
label_encoder.fit(df['label'])

# Transform actual labels to numeric labels
df['numeric_labels'] = label_encoder.transform(df['label'])

In [6]:
X = df.drop(columns=['label','numeric_labels','file_path'])
y = df['numeric_labels']

In [7]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [8]:
from collections import Counter

# Jumlah kelas sebelum SMOTE
print("Jumlah kelas sebelum SMOTE:")
print(Counter(y))

# Jumlah kelas setelah SMOTE
print("\nJumlah kelas setelah SMOTE:")
print(Counter(y_resampled))

Jumlah kelas sebelum SMOTE:
Counter({2: 3570, 39: 3177, 19: 3012, 6: 3003, 27: 2955, 41: 2784, 24: 2766, 0: 2751, 20: 2733, 37: 2691, 10: 2550, 33: 2547, 18: 2520, 21: 2484, 42: 2472, 8: 2442, 40: 2367, 26: 2331, 30: 2283, 28: 2277, 23: 2268, 32: 2238, 7: 2202, 31: 2196, 43: 2181, 15: 2073, 44: 2073, 5: 2016, 34: 1977, 9: 1962, 25: 1947, 12: 1932, 1: 1914, 16: 1890, 29: 1884, 14: 1863, 3: 1830, 17: 1830, 4: 1797, 11: 1773, 35: 1758, 36: 1752, 38: 1737, 22: 1695, 13: 1623})

Jumlah kelas setelah SMOTE:
Counter({0: 3570, 4: 3570, 9: 3570, 27: 3570, 18: 3570, 24: 3570, 23: 3570, 13: 3570, 20: 3570, 11: 3570, 16: 3570, 40: 3570, 28: 3570, 15: 3570, 21: 3570, 8: 3570, 31: 3570, 29: 3570, 5: 3570, 7: 3570, 41: 3570, 44: 3570, 3: 3570, 2: 3570, 14: 3570, 43: 3570, 36: 3570, 26: 3570, 34: 3570, 12: 3570, 37: 3570, 35: 3570, 42: 3570, 33: 3570, 17: 3570, 25: 3570, 6: 3570, 30: 3570, 39: 3570, 22: 3570, 10: 3570, 1: 3570, 19: 3570, 32: 3570, 38: 3570})


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled,
                                                    test_size=0.1, random_state=42,
                                                    stratify=y_resampled)

In [10]:
from sklearn.preprocessing import StandardScaler

# Inisialisasi StandardScaler
scaler = StandardScaler()

# Fit scaler pada data training dan transform kedua set data
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [11]:
pd.DataFrame(X_train_normalized)

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1.352238,-0.196662,0.640072,-0.284901,0.150039,0.943252,0.880910,-0.821538,-1.272965,1.999582,...,0.522169,1.053469,-0.129764,1.387112,-0.021383,-1.618474,1.568124,0.056370,1.628812,1.043047
1,0.271733,-0.665258,0.836996,0.396457,0.582783,0.882425,0.794394,-0.673104,-0.867479,-0.794859,...,1.522205,-1.406470,-0.187701,1.931407,-0.162103,-1.784205,-0.344551,-1.063729,-1.409148,-0.204198
2,0.528691,-0.644129,0.929076,1.055296,1.109258,0.378360,-0.140977,-1.046839,1.146569,0.340942,...,0.194853,1.166748,0.519591,0.958469,1.159747,0.985061,1.611397,0.022832,0.570222,1.189172
3,0.512735,-0.256270,-0.857106,0.105482,-0.708300,-0.902103,0.527255,0.835797,0.851341,-0.771835,...,1.405573,0.116354,-0.656836,-0.675212,0.942161,-0.064677,0.887759,1.558140,0.888050,1.175272
4,-0.625378,-0.666065,0.561865,0.896232,0.809715,0.270311,-0.059167,-0.685461,0.501324,-0.422861,...,-0.077567,-0.842436,-0.850719,-1.158440,-0.753182,0.093594,0.976191,1.429651,0.098162,-1.441473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144580,-0.037378,-1.007166,0.597109,0.355742,0.393019,0.707309,0.319089,-0.579387,-0.870306,-1.013493,...,0.074152,0.581089,0.705050,-0.319923,0.424395,0.154773,-0.444200,0.427482,0.071098,0.029846
144581,-2.002882,0.713715,-2.152560,-2.585330,-2.484801,-1.354048,-2.309304,1.718162,-0.822125,-0.681816,...,-1.082210,-2.144068,-0.527631,-0.414229,-1.435923,-2.879149,-0.224998,1.361394,0.811245,-0.053964
144582,-1.158550,0.150183,-0.425355,0.073529,-0.252905,-0.507049,0.858439,0.128575,-0.694644,-0.802791,...,-1.886316,-1.573437,0.208306,-2.257273,-1.005788,0.755442,-1.261920,0.448821,1.568874,-0.963251
144583,-1.487249,0.340728,-0.733062,0.409354,-0.158977,-1.183018,0.303486,0.203856,0.635147,-0.210467,...,0.709369,0.631290,0.098589,0.015874,0.338881,-0.412642,-0.521079,-0.685446,0.336596,-1.377381


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
import numpy as np

np.random.seed(42)

# Base models
rf = RandomForestClassifier(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='log2',
    max_depth=None,
    bootstrap=False,
    random_state=42
)
knn = KNeighborsClassifier(
    n_neighbors=1)

dt = DecisionTreeClassifier(
	random_state=42)

# Final estimator
final_estimator = LogisticRegression(multi_class='multinomial', max_iter=2000)

# Function to get out-of-fold predictions
def get_oof_pred(model, X, y, cv=3):
    oof_pred = cross_val_predict(model, X, y, cv=cv, method='predict_proba')
    return oof_pred

# Get out-of-fold predictions for base models
rf_oof = get_oof_pred(rf, X_train_normalized, y_train)
knn_oof = get_oof_pred(knn, X_train_normalized, y_train)
dt_oof = get_oof_pred(dt, X_train_normalized, y_train)

# Combine out-of-fold predictions
X_train_meta = np.hstack([rf_oof, knn_oof, dt_oof])

# Fit final estimator
final_estimator.fit(X_train_meta, y_train)

# Fit base models on entire training data
rf.fit(X_train_normalized, y_train)
knn.fit(X_train_normalized, y_train)
dt.fit(X_train_normalized, y_train)

# Get predictions for test data
rf_test = rf.predict_proba(X_test_normalized)
knn_test = knn.predict_proba(X_test_normalized)
dt_test = dt.predict_proba(X_test_normalized)

# Combine test predictions
X_test_meta = np.hstack([rf_test, knn_test, dt_test])

# Final prediction
final_pred = final_estimator.predict(X_test_meta)

# Calculate accuracy
accuracy = accuracy_score(y_test, final_pred)
print(f"Accuracy of the stacking model: {accuracy:.5f}")

Accuracy of the stacking model: 0.89518


In [13]:
from sklearn.metrics import accuracy_score, classification_report

# Generate and print classification report
print("\nClassification Report:")
print(classification_report(y_test, final_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.88      0.90       357
           1       0.86      0.90      0.88       357
           2       0.92      0.87      0.90       357
           3       0.88      0.87      0.88       357
           4       0.89      0.88      0.88       357
           5       0.88      0.85      0.86       357
           6       0.92      0.76      0.84       357
           7       0.91      0.94      0.93       357
           8       0.90      0.87      0.89       357
           9       0.93      0.95      0.94       357
          10       0.88      0.87      0.87       357
          11       0.86      0.86      0.86       357
          12       0.86      0.89      0.87       357
          13       0.89      0.94      0.91       357
          14       0.89      0.88      0.88       357
          15       0.90      0.94      0.92       357
          16       0.94      0.97      0.96       357
   